In [1]:
import os
import sys
import requests

import numpy as np
import pandas as pd

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [2]:
np.random.seed(42)


In [3]:
df_path = os.path.join(os.getcwd(), os.pardir, 'data', 'data.csv')
df = pd.read_csv(df_path)

features = ['acousticness', 'danceability','energy',
               'instrumentalness', 'key', 'liveness', 'loudness',
               'mode','speechiness', 'tempo',
               'valence']
df_train = df[features]

df_train.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,0.995,0.708,0.1950,0.563,10,0.1510,-12.428,1,0.0506,118.469,0.7790
1,0.994,0.379,0.0135,0.901,8,0.0763,-28.454,1,0.0462,83.972,0.0767
2,0.604,0.749,0.2200,0.000,5,0.1190,-19.924,0,0.9290,107.177,0.8800
3,0.995,0.781,0.1300,0.887,1,0.1110,-14.734,0,0.0926,108.003,0.7200
4,0.990,0.210,0.2040,0.908,11,0.0980,-16.829,1,0.0424,62.149,0.0693


In [4]:
scaler = StandardScaler()
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train),
                               columns=features)
df_train_scaled.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,1.332319,0.968662,-1.097999,1.296562,1.365333,-0.314998,-0.186652,0.641344,-0.289840,0.049500,0.940924
1,1.329664,-0.907636,-1.776785,2.389253,0.796383,-0.737519,-3.014729,0.641344,-0.319186,-1.073199,-1.735454
2,0.294154,1.202486,-1.004503,-0.523513,-0.057043,-0.495997,-1.509457,-1.559227,5.568626,-0.317996,1.325822
3,1.332319,1.384983,-1.341091,2.343994,-1.194943,-0.541247,-0.593587,-1.559227,-0.009722,-0.291114,0.716082
4,1.319044,-1.871449,-1.064341,2.411883,1.649808,-0.614778,-0.963288,0.641344,-0.344530,-1.783425,-1.763655


### Autoencoder

In [5]:
n = df_train_scaled.shape[1]

# Encoder

encoder = Sequential([Dense(n, name='encode_1', input_shape=(n,)),
                      Dense(n // 1.25, name='encode_2'),
                      Dense(n // 2, name='encode_3')])

encoder.compile(optimizer='adam', loss='mse')
encoder.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encode_1 (Dense)             (None, 11)                132       
_________________________________________________________________
encode_2 (Dense)             (None, 8)                 96        
_________________________________________________________________
encode_3 (Dense)             (None, 5)                 45        
Total params: 273
Trainable params: 273
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Decoder
decoder = Sequential([Dense(n // 2, name='decode_1', input_shape=(n // 2,)),
                      Dense(n // 1.25, name='decode_2'),
                      Dense(n, name='decode_3')])

decoder.compile(optimizer='adam', loss='mse')
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decode_1 (Dense)             (None, 5)                 30        
_________________________________________________________________
decode_2 (Dense)             (None, 8)                 48        
_________________________________________________________________
decode_3 (Dense)             (None, 11)                99        
Total params: 177
Trainable params: 177
Non-trainable params: 0
_________________________________________________________________


### Build the full autoencoder and train it

In [7]:
input_layer = Input(shape=(n,))
encoder_output = encoder(input_layer)
decoder_output = decoder(encoder_output)
autoencoder = Model(input_layer, decoder_output)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
sequential (Sequential)      (None, 5)                 273       
_________________________________________________________________
sequential_1 (Sequential)    (None, 11)                177       
Total params: 450
Trainable params: 450
Non-trainable params: 0
_________________________________________________________________


In [8]:
stop = EarlyStopping(monitor='loss', 
                     patience=5,
                     min_delta=0.0001,
                     restore_best_weights=True)

history = autoencoder.fit(df_train_scaled, 
                          df_train_scaled, 
                          epochs=200, 
                          batch_size=128,
                          callbacks=[stop])

Train on 169909 samples
Epoch 1/200
169909/169909 [==============================] - 3s 17us/sample - loss: 0.4533
Epoch 2/200
169909/169909 [==============================] - 3s 17us/sample - loss: 0.3070
Epoch 3/200
169909/169909 [==============================] - 3s 16us/sample - loss: 0.3046
Epoch 4/200
169909/169909 [==============================] - 3s 16us/sample - loss: 0.3039
Epoch 5/200
169909/169909 [==============================] - 3s 17us/sample - loss: 0.3038
Epoch 6/200
169909/169909 [==============================] - 3s 17us/sample - loss: 0.3038
Epoch 7/200
169909/169909 [==============================] - 3s 17us/sample - loss: 0.3038
Epoch 8/200
169909/169909 [==============================] - 3s 17us/sample - loss: 0.3038
Epoch 9/200
169909/169909 [==============================] - 3s 17us/sample - loss: 0.3038
Epoch 10/200
169909/169909 [==============================] - 3s 17us/sample - loss: 0.3038


### Save the trained models

In [9]:
encoder.save('encoder.h5')
decoder.save('decoder.h5')
autoencoder.save('autoencoder.h5')

### Build an end to end model with encoder and nearest neighbors

We are stacking a keras NN encoder on top of a scikit-learn nearest neighbors model. The former reduces the dimensionality, from 11 to 5 features. 

In [10]:
class End2EndModel():
    def __init__(self, n_examples, encoder_file):
        self.scaler = StandardScaler()
        self.encode = load_model(encoder_file)

        self.nearest_n = NearestNeighbors(n_examples)
        
    def fit(self, X):
        X_scaled = self.scaler.fit_transform(X)
        encoded = self.encode.predict(X_scaled)
        nn = self.nearest_n.fit(encoded)
        return nn
    
    def predict(self, x):
        x_scaled = self.scaler.transform(x)
        encoded = self.encode.predict(x_scaled)
        scores, indices = self.nearest_n.kneighbors(encoded)
        return scores, indices
                

In [11]:
model = End2EndModel(10, 'encoder.h5')

C:\Users\A\AppData\Local\Continuum\anaconda3\envs\song-recommend\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [12]:
model.fit(df_train)

NearestNeighbors(n_neighbors=10)

### Try it out with an item from the dataset

In [13]:
test = np.array(df_train.iloc[20215])
test = test.reshape(1, -1)
test


array([[ 2.03000e-01,  6.09000e-01,  4.43000e-01,  1.04000e-03,
         1.10000e+01,  1.83000e-01, -1.14780e+01,  0.00000e+00,
         3.05000e-02,  1.22792e+02,  2.11000e-01]])

In [14]:
scores, indices = model.predict(test)
scores, indices

(array([[2.16066837e-07, 1.37944102e-01, 1.73460015e-01, 2.04828354e-01,
         2.12524099e-01, 2.71176551e-01, 2.72602978e-01, 2.72674586e-01,
         2.84520836e-01, 3.00581317e-01]]),
 array([[ 20215,   2647,   5257, 157834, 159982, 104550,  82920,  39635,
         138843,  66111]], dtype=int64))

### Retrieve our suggestions

In [15]:
results = []
for i, index in enumerate(indices[0]):
    track_id = df.iloc[index]['id']
    artists = df.iloc[index]['artists']
    title = df.iloc[index]['name']
    results.append({'index': index,
                    'track_id': track_id,
                    'artists': artists,
                    'title': title,
                    'score': scores[0][i]})
    
result_table = pd.DataFrame(results)
result_table.sort_values(by='score')

,index,track_id,artists,title,score
0,20215,4nTXzIW8EjH0V1NBxyhatX,['Grateful Dead'],Lost Sailor - 2013 Remaster,2.160668e-07
1,2647,66ark8uqwtus4LkRSTn8UG,['Chad & Jeremy'],Before and After,1.379441e-01
2,5257,2PAol2oDdGSHys8hc0gtLX,['Tori Amos'],Precious Things,1.734600e-01
3,157834,7EtOFVnHpg6Czxb7pwG2j0,['Amanda Miguel'],Dudas,2.048284e-01
4,159982,0mqBx2unSAs6w8qHWDHdC7,['Shinedown'],Save Me - Acoustic,2.125241e-01
5,104550,2zyTP97uGsIc1C4KNNEkyn,['Bobby Womack'],Across 110th Street,2.711766e-01
6,82920,5aHHf6jrqDRb1fcBmue2kn,['The Beatles'],The End - Remastered 2009,2.726030e-01
7,39635,49ONd7q61KYjSZE0A8gtCW,['K CAMP'],Blessing,2.726746e-01
8,138843,2Pdh7gm93N9GK8jkBbMIvb,['Frank Sinatra'],Call Me,2.845208e-01
9,66111,6JWLeCDXGkCFlB6aIDNsCF,['Jethro Tull'],Minstrel in the Gallery - 2002 Remaster,3.005813e-01


### Explore ways to persist the scikit-learn layer

In [16]:
import pickle
with open('nearest.pickle', 'wb') as f:
    pickle.dump(model.nearest_n, f)